In [9]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd
import os 

import lightgbm as lgb
import xgboost as xgb 
import catboost as cbt 

FOLDER_DIR = "./preprocessed/"

In [14]:
train_df = pd.read_csv(os.path.join(FOLDER_DIR,'train_df.csv'))
test_df = pd.read_csv(os.path.join(FOLDER_DIR,'test_df.csv'))
X = train_df.drop(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'scalar_coupling_constant'], axis=1)
X_test = test_df.drop(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'scalar_coupling_constant'], axis=1)
Y_test = test_df['scalar_coupling_constant']
Y = train_df['scalar_coupling_constant']
X

,x0,y0,z0,x1,y1,z1,dist,dist_x,dist_y,dist_z,1JHC,1JHN,2JHC,2JHH,2JHN,3JHC,3JHH,3JHN
0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,0.014849,1.091835,0.006025,True,False,False,False,False,False,False,False
1,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,1.009580,1.469782,0.001700,False,False,False,True,False,False,False,False
2,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,0.542965,1.453558,0.878620,False,False,False,True,False,False,False,False
3,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,0.525964,1.443964,0.904421,False,False,False,True,False,False,False,False
4,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,1.024429,0.377947,0.007724,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3724006,-0.254709,0.115179,-2.320941,1.396720,-0.764049,0.445825,3.339948,1.651428,0.879228,2.766766,False,False,False,False,False,True,False,False
3724007,-0.254709,0.115179,-2.320941,0.709493,0.478235,0.961172,3.440023,0.964202,0.363056,3.282113,False,False,False,False,False,True,False,False
3724008,-0.254709,0.115179,-2.320941,-0.302582,0.693610,-0.181481,2.216791,0.047874,0.578431,2.139460,False,False,True,False,False,False,False,False
3724009,-0.254709,0.115179,-2.320941,0.402259,0.359544,-1.482579,1.092782,0.656968,0.244365,0.838362,True,False,False,False,False,False,False,False


### Boosting

In [ ]:
N_fold = 5
index = np.arange(len(X))

models = []

def train(model_dict, i, modelname='lgb'):
    print("-------------------------")
    model = model_dict[modelname]
    print(model)
    model.fit(X[index%N_fold!=i], Y[index%N_fold!=i], 
                eval_set=[(X[index%N_fold==i], Y[index%N_fold==i])], 
                verbose=2, 
                early_stopping_rounds=100
                )
    models.append(model)
    return 

model_dict = {
    'lgb': lgb.LGBMRegressor(objective='regression_l1', n_estimators=200, learning_rate=0.01, max_depth=2,num_leaves=35),
    'xgb': xgb.XGBRegressor(tree_method='hist', objective='reg:absoluteerror', n_estimators=100, max_depth=5, num_leaves=10),
    'cbt': cbt.CatBoostRegressor(objective='MAE', iterations=200, max_depth=5, learning_rate=0.01),
}

for i in range(N_fold):
    train(model_dict, i, 'xgb')
    # train(model_dict, i, 'lgb')
    # train(model_dict, i, 'cbt')

### Simple RandomForest

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, Y)

## Evaluation

In [17]:
def log_mae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    log_mae = np.log(mae)
    return log_mae

In [20]:
y_pred = np.mean([model.predict(X_test) for model in models], axis=0)
    
print("Score :", log_mae(Y_test, y_pred))

/Users/gauthieravite/anaconda3/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/gauthieravite/anaconda3/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/gauthieravite/anaconda3/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/gauthieravite/anaconda3/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  r

Score : 0.8612645499358786
